<a href="https://colab.research.google.com/github/odhiambk/CIS-678-Machine-Learning-Project-4/blob/main/AI4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam

class DietRecommendationAgent:
    def __init__(self, dataset_path):
        """
        Initialize the AI agent with the recipe dataset.
        """
        self.dataset = pd.read_csv(dataset_path)
        self.preprocessed_data = None
        self.autoencoder_model = None
        self.scaler = MinMaxScaler()

    def preprocess_data(self):
        """
        Preprocess the dataset by selecting relevant columns and scaling the data.
        """
        selected_columns = ['Calories', 'FatContent', 'SaturatedFatContent',
                          'CholesterolContent', 'SodiumContent', 'CarbohydrateContent',
                          'FiberContent', 'SugarContent', 'ProteinContent']
        df = self.dataset[selected_columns].dropna()
        self.preprocessed_data = self.scaler.fit_transform(df)
        print("Data preprocessing complete.")

    def build_autoencoder(self):
        """
        Build and compile the autoencoder model.
        """
        num_features = self.preprocessed_data.shape[1]
        model = Sequential([
            Dense(256, input_shape=(num_features,)),
            LeakyReLU(),
            Dropout(0.2),
            Dense(128),
            LeakyReLU(),
            Dropout(0.2),
            Dense(64),
            LeakyReLU(),
            Dropout(0.2),
            Dense(32),  # Bottleneck layer
            Dense(64),
            LeakyReLU(),
            Dropout(0.2),
            Dense(128),
            LeakyReLU(),
            Dropout(0.2),
            Dense(256),
            LeakyReLU(),
            Dense(num_features, activation='linear')
        ])
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])
        self.autoencoder_model = model
        print("Autoencoder model built.")

    def train_autoencoder(self, epochs=10):
        """
        Train the autoencoder model on the preprocessed data.
        """
        X_train = self.preprocessed_data
        self.autoencoder_model.fit(X_train, X_train, epochs=epochs, validation_split=0.2)
        print("Autoencoder training complete.")

    def recommend_meals_nn(self, user_preferences, top_n=5):
        """
        Recommend meals using the neural network-based approach.
        """
        # Predict latent features
        latent_features = self.autoencoder_model.predict(self.preprocessed_data)
        user_vector = np.array([user_preferences])
        user_vector_scaled = self.scaler.transform(user_vector)
        user_latent = self.autoencoder_model.predict(user_vector_scaled)

        # Compute cosine similarity
        similarity_scores = cosine_similarity(user_latent, latent_features)
        top_indices = similarity_scores.argsort()[0][::-1][:top_n]
        recommendations = self.dataset.iloc[top_indices]
        return recommendations

    def recommend_meals_caloric(self, user_profile, top_n=5):
        """
        Recommend meals based on caloric needs.
        """
        bmr = self.calculate_bmr(user_profile)
        caloric_target = bmr * user_profile['activity_level']
        self.dataset['caloric_diff'] = abs(self.dataset['Calories'] - caloric_target)
        recommendations = self.dataset.nsmallest(top_n, 'caloric_diff')
        return recommendations

    @staticmethod
    def calculate_bmr(user_profile):
        """
        Calculate Basal Metabolic Rate (BMR) using the Harris-Benedict equation.
        """
        if user_profile['gender'] == 'male':
            bmr = 88.362 + (13.397 * user_profile['weight']) + (4.799 * user_profile['height']) - (5.677 * user_profile['age'])
        else:
            bmr = 447.593 + (9.247 * user_profile['weight']) + (3.098 * user_profile['height']) - (4.330 * user_profile['age'])
        return bmr

    def run_agent(self, user_profile, approach='nn', top_n=5):
        """
        Run the AI agent to generate meal recommendations.
        """
        if approach == 'nn':
            print("Using neural network-based recommendation approach...")
            recommendations = self.recommend_meals_nn(user_profile, top_n)
        elif approach == 'caloric':
            print("Using caloric-based recommendation approach...")
            recommendations = self.recommend_meals_caloric(user_profile, top_n)
        else:
            raise ValueError("Invalid approach. Choose 'nn' or 'caloric'.")
        return recommendations

# Instance
if __name__ == "__main__":
    dataset_path = "recipes.csv"
    agent = DietRecommendationAgent(dataset_path)

    # Preprocess data and build the model
    agent.preprocess_data()
    agent.build_autoencoder()
    agent.train_autoencoder(epochs=10)

    # User profile example
    user_profile = {
        'gender': 'female',
        'age': 30,
        'weight': 65,  # in kg
        'height': 165,  # in cm
        'activity_level': 1.2  # Sedentary
    }

    # Get recommendations
    recommendations = agent.run_agent(user_profile, approach='caloric', top_n=5)
    print(recommendations)
